# Adım 6: Veri Kalite Testleri (Great Expectations)

**Amaç:** `silver_bookings` tablosunun kalitesini doğrulamak.

| Test | Açıklama |
|------|----------|
| Completeness | Kritik kolonlarda NULL yok |
| Uniqueness | `booking_id` benzersiz |
| Validity | Fiyatlar ≥ 0, tarihler geçerli |
| Type Check | Kolom tipleri doğru |

**Not:** GX kurulumu için kernel restart gerekebilir.

In [1]:
%pip install great_expectations==0.18.21 --quiet

StatementMeta(, a0b9fc83-b212-4f63-a649-4b3bf9b93302, 7, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [2]:
import great_expectations as ge
from great_expectations.dataset import SparkDFDataset
import pyspark.sql.functions as F

print(f"✅ Great Expectations version: {ge.__version__}")

StatementMeta(, a0b9fc83-b212-4f63-a649-4b3bf9b93302, 9, Finished, Available, Finished)

<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


## 1. Veriyi Yükle

In [3]:
df_silver = spark.read.table("silver_bookings")
gx_df = SparkDFDataset(df_silver)

print(f"✅ silver_bookings yüklendi. Satır sayısı: {df_silver.count()}")
print(f"Kolonlar: {df_silver.columns}")

StatementMeta(, a0b9fc83-b212-4f63-a649-4b3bf9b93302, 10, Finished, Available, Finished)

✅ silver_bookings yüklendi. Satır sayısı: 1338272
Kolonlar: ['hotel_id', 'hotel_name', 'country', 'city', 'hotel_type', 'star_rating', 'total_rooms', 'hotel_facilities', 'hotel_description', 'nearby_attractions', 'latitude', 'longitude', 'website', 'customer_id', 'first_name', 'last_name', 'full_name', 'email', 'phone', 'birth_date', 'gender', 'country_customer', 'city_customer', 'address', 'postal_code', 'language_preference', 'loyalty_level_customer', 'registration_date', 'marketing_consent', 'booking_id', 'booking_date', 'checkin_date', 'checkout_date', 'nights', 'adults', 'children', 'infants', 'room_type', 'rooms_booked', 'booking_channel', 'special_requests', 'is_cancelled', 'cancellation_date', 'cancellation_reason', 'total_amount', 'room_price', 'tax_amount', 'service_fee', 'payment_status', 'payment_method', 'booking_source', 'promotion_code', 'discount_amount', 'created_timestamp', 'updated_timestamp', 'created_by', 'booking_status', 'review_id', 'review_rating', 'review_titl

## 2. Expectation'ları Tanımla

In [4]:
print("🧪 A. Completeness (Not Null) Testleri...")
gx_df.expect_column_values_to_not_be_null("booking_id")
gx_df.expect_column_values_to_not_be_null("customer_id")   # FIX: client_id → customer_id
gx_df.expect_column_values_to_not_be_null("hotel_id")
gx_df.expect_column_values_to_not_be_null("booking_date")

print("🧪 B. Uniqueness Testleri...")
gx_df.expect_column_values_to_be_unique("booking_id")

print("🧪 C. Validity (Fiyat ≥ 0) Testleri...")
gx_df.expect_column_values_to_be_between("total_amount", min_value=0)
gx_df.expect_column_values_to_be_between("room_price", min_value=0)

print("🧪 D. Type Testleri...")
gx_df.expect_column_values_to_be_of_type("booking_id", "StringType")
gx_df.expect_column_values_to_be_of_type("total_amount", "DoubleType")
gx_df.expect_column_values_to_be_of_type("customer_id", "StringType")

print("🧪 E. Domain Testleri...")
gx_df.expect_column_values_to_be_in_set(
    "currency",
    ["EUR", "USD", "GBP", "JPY", "TRY", "AED", "CNY"]
)

print("✅ Tüm expectation'lar tanımlandı.")

StatementMeta(, a0b9fc83-b212-4f63-a649-4b3bf9b93302, 11, Finished, Available, Finished)

🧪 A. Completeness (Not Null) Testleri...
🧪 B. Uniqueness Testleri...
🧪 C. Validity (Fiyat ≥ 0) Testleri...
🧪 D. Type Testleri...
🧪 E. Domain Testleri...
✅ Tüm expectation'lar tanımlandı.


## 3. Validasyon Çalıştır & Rapor

In [5]:
print("📊 Full Validation Suite çalışıyor...")
results = gx_df.validate()

# Özet
total_tests  = len(results["results"])
passed_tests = sum(1 for r in results["results"] if r["success"])
failed_tests = total_tests - passed_tests

print(f"\n{'='*50}")
print(f"📋 VALIDATION REPORT — silver_bookings")
print(f"{'='*50}")
print(f"  Toplam Test  : {total_tests}")
print(f"  ✅ Geçti     : {passed_tests}")
print(f"  ❌ Kaldı     : {failed_tests}")
print(f"{'='*50}")

if results["success"]:
    print("🎉 SUCCESS! Tüm kalite kontrolleri geçti.")
else:
    print("⚠️ Bazı testler başarısız:")
    for res in results["results"]:
        status = "✅" if res["success"] else "❌"
        exp_type = res["expectation_config"]["expectation_type"]
        col_name = res["expectation_config"]["kwargs"].get("column", "")
        print(f"  {status} {exp_type}({col_name})")
        if not res["success"] and "result" in res:
            print(f"       → {res['result']}")

StatementMeta(, a0b9fc83-b212-4f63-a649-4b3bf9b93302, 12, Finished, Available, Finished)

📊 Full Validation Suite çalışıyor...

📋 VALIDATION REPORT — silver_bookings
  Toplam Test  : 11
  ✅ Geçti     : 9
  ❌ Kaldı     : 2
⚠️ Bazı testler başarısız:
  ✅ expect_column_values_to_not_be_null(booking_id)
  ✅ expect_column_values_to_be_unique(booking_id)
  ✅ expect_column_values_to_be_of_type(booking_id)
  ❌ expect_column_values_to_not_be_null(customer_id)
       → {'element_count': 1338272, 'unexpected_count': 8012, 'unexpected_percent': 0.5986824800937328, 'unexpected_percent_total': 0.5986824800937328, 'partial_unexpected_list': [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]}
  ✅ expect_column_values_to_be_of_type(customer_id)
  ❌ expect_column_values_to_not_be_null(hotel_id)
       → {'element_count': 1338272, 'unexpected_count': 7895, 'unexpected_percent': 0.5899398627483801, 'unexpected_percent_total': 0.5899398627483801, 'partial_unexpected_list': [None, None, None, None, None, None, None, None, None

## 4. Native PySpark Kalite Özeti

GX kurulumu başarısız olursa bu cell'i çalıştır.

In [6]:
from pyspark.sql.functions import col, count, when, isnan, isnull

df = spark.read.table("silver_bookings")
total = df.count()

checks = [
    ("booking_id NULL",    df.filter(col("booking_id").isNull()).count()),
    ("customer_id NULL",   df.filter(col("customer_id").isNull()).count()),
    ("hotel_id NULL",      df.filter(col("hotel_id").isNull()).count()),
    ("booking_date NULL",  df.filter(col("booking_date").isNull()).count()),
    ("total_amount < 0",   df.filter(col("total_amount") < 0).count()),
    ("city_clean NULL",    df.filter(col("city_clean").isNull()).count()),
    ("duplicate booking",  total - df.select("booking_id").distinct().count()),
]

print(f"\n{'='*55}")
print(f"{'TEST':<30} {'SORUNLU':>10} {'DURUM':>10}")
print(f"{'='*55}")
for name, count_val in checks:
    status = "✅ OK" if count_val == 0 else f"❌ {count_val:,}"
    print(f"{name:<30} {count_val:>10,} {status:>10}")
print(f"{'='*55}")
print(f"Toplam satır: {total:,}")

StatementMeta(, a0b9fc83-b212-4f63-a649-4b3bf9b93302, 13, Finished, Available, Finished)


TEST                              SORUNLU      DURUM
booking_id NULL                         0       ✅ OK
customer_id NULL                    8,012    ❌ 8,012
hotel_id NULL                       7,895    ❌ 7,895
booking_date NULL                       0       ✅ OK
total_amount < 0                        0       ✅ OK
city_clean NULL                     3,910    ❌ 3,910
duplicate booking                       0       ✅ OK
Toplam satır: 1,338,272
